In [1]:
!pip install --quiet langchain langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00


In [24]:
import os

os.environ["OPENAI_API_TYPE"] = "azure_ad"
os.environ["OPENAI_API_VERSION"] = "2024-05-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = "This form the Azure portal - model deployment"
os.environ["AZURE_OPENAI_API_KEY"] = 'This form the Azure portal - model deployment'



In [25]:
from langchain_openai import AzureChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [26]:
llm = AzureChatOpenAI(
            temperature=1,
            azure_deployment="gpt-4o",
        )

 When we have to pass multiput input to a chain, we cant use simple sequentail chain. In this case we will use a Complex sequential chain.

## Chain 1: Synopsis Chain.

In [38]:
template = """
  You are a playwright. Given the title of play and the era it is set in,
  it is your job to write a short synopsis for that title.

  Title: {title}
  Era: {era}

  Playwright: This is a synopsis for the above play:
"""

prompt_tempalte = PromptTemplate(
    input_variables=['title', 'era'],
    template=template
)

In [39]:
synopsis_chain = LLMChain(llm=llm,
                          prompt=prompt_tempalte,
                          output_key = 'synopsis')

## Chain 2: Review Chain.

In [40]:
template = """
  You are a play critic from the New York Times. Given the synopsis of play,
  it is your job to write a short review for that play.

  Play Synopsis:
  {synopsis}

  Review from a New York Times play critic of the above play:
"""

prompt_tempalte = PromptTemplate(
    input_variables=['synopsis'],
    template=template
)

In [30]:
review_chain = LLMChain(llm=llm,
                        prompt=prompt_tempalte,
                        output_key='review')


## Chain 3: Social Media Chain.

In [48]:
template = """
  You are a social media manager for a theater company.  Given the title of play,
  the era it is set in, and the review of the play, it is your job to write a short social media post for that play.

  Play Synopsis:
  {synopsis}

  Review from a New York Times play critic of the above play:
  {review}

  Social Media Post:
"""


prompt_template = PromptTemplate(
    input_variables=["synopsis", "review"],
    template=template
                                 )


In [49]:
social_chain = LLMChain(llm=llm,
                        prompt=prompt_template,
                        output_key="social_post_text")

# Sequential Chain

In [45]:
from langchain.chains import SequentialChain

In [50]:
overall_chain = SequentialChain(
    chains = [synopsis_chain, review_chain, social_chain],
    input_variables=['title', 'era'],
    output_variables = ['social_post_text'],
    verbose = True
)

In [51]:
overall_chain.invoke({"title":"Tragedy at sunset on the beach", "era": "Victorian England"})



> Entering new SequentialChain chain...

> Finished chain.


{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'social_post_text': '🌅✨ Step back into the strict yet stunning world of Victorian England with "Tragedy at Sunset on the Beach." This poignant tale of forbidden love, societal constraints, and heart-wrenching choices will transport you to a time where the cost of defiance was steep. \n\n✨ The New York Times hails it as "a deeply moving exploration of love\'s defiance against the cruelty of societal constraints," with performances that breathe life into the eternal struggle between duty and desire.\n\n🎭 Join Lady Evelyn and Thomas Cavendish as their secret seaside romance faces the ultimate test under the fading light of dusk. Will love conquer all, or will societal expectations cast a shadow on their dreams forever?\n\nDon\'t miss this evocative production that promises to resonate with modern audiences. \n\n#TragedyAtSunsetOnTheBeach #VictorianDrama #ForbiddenLove #TheaterMagic #MustSeeStage #EmotionalJourney #T